In [ ]:
import pandas as pd
import sklearn
from sklearn import svm
from sklearn.metrics import roc_auc_score
import numpy as np
from sklearn.model_selection import cross_validate,cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
import pprint as pp
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm
from sklearn.linear_model import LogisticRegression
import xgboost as xgb

df=pd.read_csv('liver.csv')
df=df.fillna(0)

# 1

In [ ]:
model=svm.SVC(random_state=1234)
X=df.drop('category',axis=1)
y=df['category']


svc_score=cross_val_score(model,X,y,cv=10,scoring='roc_auc')
print(f"fold scores are {svc_score}")
print(f"mean score {np.mean(svc_score)}")


# 2

In [ ]:
n_estimators= [int(x) for x in np.linspace(start = 200, stop = 2000,num = 10)]


# Maximum number of levels in tree
max_depth= [int(x) for x in np.linspace(10, 110, num = 11)]

# Minimum number of samples required to split a node
min_samples_split= [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf= [1, 2, 4]# Method of selecting samples for training each tree
bootstrap = [True, False]

random_grid= {'n_estimators': n_estimators,'max_depth': max_depth,'min_samples_split': min_samples_split,'min_samples_leaf': min_samples_leaf,'bootstrap': bootstrap}
pp.pprint(random_grid)
rf = RandomForestClassifier()
rf_random= RandomizedSearchCV(estimator = rf, param_distributions= random_grid, n_iter= 100, cv = 10, verbose=2, random_state=1234, n_jobs= -1)
rf_random.fit(X,y)

In [ ]:


best_random_model= rf_random.best_estimator_
print(best_random_model)


In [ ]:
best_random_scores= cross_val_score(best_random_model,X,y,cv=10,scoring='f1')
best_random_accuracy= np.mean(best_random_scores)

print(f"fold scores are {best_random_scores}")
print(f"mean score {best_random_accuracy}")


# 3

In [ ]:
model1=LogisticRegression()
model2=DecisionTreeClassifier()
model3=RandomForestClassifier()
model4=svm.SVC()

models=[model1,model2,model3,model4]
names=['Logistic','Decisiontree','RandomForest','SVC']

results=[]
for index,model in enumerate(models):
    result=cross_val_score(model,X,y,cv=10,scoring='roc_auc')
    results.append(result)
    print(f"model: {names[index]}, model result: {result}\n")

import matplotlib.pyplot as plt
fig = plt.figure()
fig.suptitle('Algorithm Comparison')
ax = fig.add_subplot(111)
plt.boxplot(results)
ax.set_xticklabels(names)
plt.show()


# 4

In [ ]:

model=xgb.XGBClassifier()


model_score= cross_validate(model,X,y,cv=10,scoring=['accuracy','balanced_accuracy','roc_auc','f1'])

print(f"xgboost accuracy is {np.mean(model_score['test_accuracy'])}")
print(f"xgboost balanced_accuracy is {np.mean(model_score['test_balanced_accuracy'])}")
print(f"xgboost roc_auc is {np.mean(model_score['test_roc_auc'])}")
print(f"xgboost f1 is {np.mean(model_score['test_f1'])}")